# 基础知识

# Socket
* socket可以在不同的电脑间通信，也可以在同一电脑的不同程序间通信

## 发送数据

In [1]:
#步骤：

#1.创建socket并连接
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#AF_INET:表示这个socket用来网络连接
#SOCK_DGRAM:表示连接是一个 udp 连接

#2.发送数据
#s.sendto(data,address)
#data:要发送的数据，二进制
#address:发送给谁，参数是一个元组，元组里有两个元素，第一个表示目标的ip地址，第二个表示程序的端口号
#端口号：0~65536    0~1024不要用，系统一些重要的服务在使用    端口号要空闲
s.sendto('hello'.encode('utf8'), ('192.168.50.220', 9000))

#3.关闭socket
s.close()

## 接收数据

In [ ]:
import socket

#创建一个基于 udp 的网络socket连接
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#绑定端口号和ip地址
s.bind(('192.168.50.220', 9000))
#recvfrom接收数据
data, addr = s.recvfrom(1024)
#接收到的数据是一个元组，第0个元素是接收到的数据，第1个元素是发送方的端口号和ip地址
print('从{}地址{}端口号接收到了消息，内容是：{}'.format(addr[0], addr[1], data.decode('utf8')))
s.close()

## TCP客户端

In [ ]:
import socket

#基于TCP协议的socket连接
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#在发送数据之前，必须要和服务器建立连接
s.connect(('192.168.50.220', 9000))
s.send('hello'.encode('utf8'))
s.close()

## TCP服务端

In [ ]:
import socket

#创建一个socket连接
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('192.168.50.220', 9000))
s.listen(128)  #把socket变成一个被动监听的
#128:允许128个排队
client_socket, client_addr = s.accept()  #接收客户端的请求
#接收到的是一个元组，有两个元素
#第0个元素是客户端的socket连接，第1个元素是客户端的ip地址和端口号
x = client_socket.recv(1024)
print('接收到了{}客户端{}端口号发送的数据，内容是{}'.format(client_addr[0], client_addr[1], x.decode('utf8')))

## 文件下载

In [ ]:
#客户端
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('192.168.50.220', 9000))
file_name = input('请输入要下载的文件名：')
s.send(file_name.encode('utf8'))
with open(file_name, 'wb') as file:
    while 1:
        content = s.recv(1024)
        if not content:
            break
        file.write(content)
s.close()

In [ ]:
#服务器
import socket, os

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('192.168.50.220', 9000))
server_socket.listen(128)
#接收客户端的请求
client_socket, client_addr = server_socket.accept()
file_name = client_socket.recv(1024).decode('utf8')
if os.path.isfile():
    with open(data, 'rb') as file:
        content = file.read()
        client_socket.send(content)
else:
    print('文件不存在')

# 多线程

In [ ]:
import threading


def dance():
    for i in range(50):
        time.sleep(0.2)
        print('我正在跳舞')


def sing():
    for i in range(50):
        time.sleep(0.2)
        print('我正在唱歌')


#多任务执行：多线程、多进程、多线程+多进程

#target 需要的是一个函数，用来指定线程需要执行的任务
t1 = threading.Thread(target=dance)  #创建了线程1
t2 = threading.Thread(target=sing)  #创建了线程2

#启动线程
t1.start()
t2.start()

## 多线程聊天

In [ ]:
import socket, threading

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.bind(('192.168.50.220', 8000))


def send_message():
    while 1:
        message = input('请输入要发送的内容：')
        s.sendto(message.encode('utf8'), ('192.168.50.220', 9000))
        if message == 'exit':
            break


def recv_message():
    while 1:
        data, addr = s.recvfrom(1024)
        print('接收到了{}地址{}端口的消息:{}'.format(addr[0], addr[1], data.decode('utf8')),
              file=open('消息记录.txt', 'a', encoding='utf8'))


threading.Thread(target=send_message).start()
threading.Thread(target=recv_message).start()
s.close()

## 多线程开发

In [ ]:
import threading

#多个线程可以同时处理一个全局变量（共享全局变量）
ticket = 20


def sell():
    global ticket
    while 1:
        if ticket > 0:
            time.sleep(1)
            ticket -= 1
            print('{}卖出一张票，还剩{}张'.format(threading.current_thread().name, ticket))
        else:
            print('票卖完了')
            break


t1 = threading.Thread(target=sell, name='线程1')
t2 = threading.Thread(target=sell, name='线程2')
t1.start()
t2.start()

## 线程锁
* 同步：当多线程共享数据时，需要进行同步控制，按预定的先后次序运行，保证多个线程安全访问竞争资源，简单的同步机制是互斥锁

In [ ]:
import threading

ticket = 20
lock = threading.Lock()


def sell():
    global ticket
    while 1:
        lock.acquire()  #加同步锁
        if ticket > 0:
            time.sleep(1)
            ticket -= 1
            lock.release()
            print('{}卖出一张票，还剩{}张'.format(threading.current_thread().name, ticket))
        else:
            lock.release()
            print('票卖完了')
            break


t1 = threading.Thread(target=sell, name='线程1')
t2 = threading.Thread(target=sell, name='线程2')
t1.start()
t2.start()

## 线程间通信

In [ ]:
#queue结构可以在不同线程间传递数据
import queue, threading

q1 = queue.Queue()


def produce():
    for i in range(20):
        time.sleep(1)
        print('生产了面包', i)
        q1.put('bread{}'.format(i))


def consume():
    for i in range(20):
        time.sleep(0.5)
        print('消费了面包{}'.format(q1.get()))  #q1.get()方法是阻塞的方法


p1 = threading.Thread(target=produce, name='p1')
c1 = threading.Thread(target=consume, name='c1')
p1.start()
c1.start()

In [ ]:
#queue有FIFO结构(first in first out)
#栈(stack)结构：后进先出，先进后出
import queue, threading

q1 = queue.Queue()


def produce():
    for i in range(20):
        time.sleep(0.5)
        print('{}生产了面包{}'.format(threading.current_thread().name, i))
        q1.put('面包{}'.format(i))


def consume():
    for i in range(10):
        time.sleep(1)
        print('{}消费了{}'.format(threading.current_thread().name, q1.get()))  #q1.get()方法是阻塞的方法


p1 = threading.Thread(target=produce, name='p1')
p2 = threading.Thread(target=produce, name='p2')
p3 = threading.Thread(target=produce, name='p3')
c1 = threading.Thread(target=consume, name='c1')
c2 = threading.Thread(target=consume, name='c2')
c3 = threading.Thread(target=consume, name='c3')
p1.start()
p2.start()
p3.start()
c1.start()
c2.start()
c3.start()

# 多进程

In [ ]:
import multiprocessing


def dance(n):
    for i in range(n):
        time.sleep(0.2)
        print('我正在跳舞')


def sing(m):
    for i in range(m):
        time.sleep(0.2)
        print('我正在唱歌')


#在Python中，以下代码不会执行，需作为一个文件运行时才会执行
if __name__ == '__main__':
    #args:用于函数的传参，是一个元组
    p1 = multiprocessing.Process(target=dance, args=(20,))
    p2 = multiprocessing.Process(target=sing, args=(30,))
    p1.start()
    p2.start()

## 进程共享全局变量

In [ ]:
import multiprocessing, threading

n = 100


def test():
    global n
    n += 1
    print('{}里n的值是{}'.format(os.getpid(), n))


def demo():
    global n
    n += 1
    print('{}里n的值是{}'.format(os.getpid(), n))


t1 = threading.Thread(target=test)
t2 = threading.Thread(target=demo)
t1.start()
t2.start()
#不同进程个子保存一份全局变量，不会共享
if __name__ == '__main__':
    p1 = multiprocessing.Process(target=test)
    p2 = multiprocessing.Process(target=demo)
    p1.start()  #==>73240里n的值是101
    p2.start()  #==>73240里n的值是101

## 进程间通信

In [ ]:
import multiprocessing


def produce(x):
    for i in range(10):
        time.sleep(1)
        print('生产了pid{} {}'.format(os.getpid(), i))
        x.put('pid{} {}'.format(os.getpid(), i))


def consume(x):
    for i in range(10):
        time.sleep(0.5)
        print('消费了{}'.format(x.get()))


if __name__ == '__main__':
    q = multiprocessing.Queue()
    p1 = multiprocessing.Process(target=produce, args=(q,))
    p2 = multiprocessing.Process(target=consume, args=(q,))
    p1.start()
    p2.start()

## 队列的使用

In [ ]:
import multiprocessing, queue

q1 = multiprocessing.Queue(5)  #进程间通信
q2 = queue.Queue()  #线程间通信
#创建队列时，可以指定最大长度。默认是0，表示不限
q1.put('hello')
q1.put('yes')
q1.put('no')
q1.put('world')
q1.put('he')
#q1.put('she')==>程序不会停止，无法放进去

In [ ]:
import multiprocessing

q = multiprocessing.Queue(5)
q.put('hello')
q.put('yes')
q.put('no')
q.put('world')
q.put('he')
q.put('how', block=True, timeout=5)
#block=True表示阻塞，如果队列满了，就等待
#timeout：超时，等待多久后程序会出错
q.get()

In [ ]:
import multiprocessing

q = multiprocessing.Queue(5)
q.put('hello')
q.put('yes')
q.put('no')
q.put('world')
q.put('he')
q.put_nowait('she')  #<==>q.put('she',block=False)
q.get()

In [ ]:
import multiprocessing

q = multiprocessing.Queue(5)
q.put('hello')
q.put('yes')
q.put('no')
q.put('world')
q.put('he')
print(q.get())
print(q.get())
print(q.get())
print(q.get())
print(q.get())
q.get(block=True, timeout=5)  #q.get_nowait()

## 进程池
* 使用大量进程

In [ ]:
import multiprocessing, os, random


def worker(msg):
    t_start = time.time()
    print('{}开始执行，进程号为{}'.format(msg, os.getpid()))
    time.sleep(random.random() * 2)  #random.random():随机生成0-1间的浮点数
    t_stop = time.time()
    print(msg, '执行完毕，耗时%0.2f' % (t_stop - t_start))


if __name__ == '__main__':
    p = multiprocessing.Pool(3)  #定义一个进程池，最大进程数为3
    for i in range(0, 10):
        p.apply_async(worker, (i,))
        #Pool().apply_async(要调用的目标,(传递给目标的参数元组,))
        #每次循环将会用空出来的子进程去调用目标
    print('------start------')
    p.close()
    p.join()  #等待p中所有子进程执行完成
    print('------end------')

#进程池间通信，需要multiprocessing.Manager().Queue()

## join方法的使用

In [ ]:


x = 10


def test(a, b):
    time.sleep(1)
    global x
    x = a + b


test(1, 1)
print(x)

In [ ]:
import threading

x = 10


def test(a, b):
    time.sleep(1)
    global x
    x = a + b


t = threading.Thread(target=test, args=(1, 1))
t.start()
print(x)

In [ ]:
import time, threading

x = 10


def test(a, b):
    time.sleep(1)
    global x
    x = a + b


t = threading.Thread(target=test, args=(1, 1))
t.start()
t.join()  #让主线程等待
print(x)

# 搭建HTTP服务器

In [ ]:
import socket

#HTTP服务器都是基于TCP的socket链接
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 9090))
#能通过 127.0.0.1 和 localhost 来访问
#127.0.0.1 和 0.0.0.0 都表示本机
#0.0.0.0 表示所有可用的地址
#127.0.0.1 仅能自己访问
#192.168.50.220 仅能同局域网访问
server_socket.listen(128)
while 1:
    client_socket, client_addr = server_socket.accept()
    data = client_socket.recv(1024).decode('utf8')
    #再返回内容之前需要先设置HTTP响应头
    client_socket.send('HTTP/1.1 200 OK'.encode('utf8'))
    #一个响应头就换一行
    client_socket.send('\n'.encode('utf8'))
    #所有响应头设置完成后再换行
    #返回消息
    client_socket.send('<h1 style="color:red">hello world</h1>'.encode('utf8'))

## HTTP请求头

In [ ]:
#根据不同的请求返回不同的内容
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 9090))
server_socket.listen(128)
while 1:
    client_socket, client_addr = server_socket.accept()
    data = client_socket.recv(1024).decode('utf8')
    #print('接收到{}的数据{}'.format(client_addr[0],data))
    path = ''
    if data:  #浏览器发送的数据有可能是空
        path = data.splitlines()[0].split(' ')[1]
        print('请求的路径是{}'.format(path))
    response_header = 'HTTP/1.1 200 OK\n'
    if path == '/login':
        response_body = '<h1 style="color:red">欢迎来到登录页面</h1>'
    elif path == '/register':
        response_body = '<h1 style="color:red">欢迎来到注册页面</h1>'
    elif path == '/':
        response_body = '<h1 style="color:red">欢迎来到首页</h1>'
    else:
        response_header = 'HTTP/1.1 404 Page Not Found\n'
        response_body = '<h1 style="color:red">您要查看的页面不存在！</h1>'
    response_header += 'content-type:text/html;charset=utf8\n'
    response_header += '\n'
    response = response_header + response_body
    client_socket.send(response.encode('utf8'))

In [ ]:
#面向对象的封装
import socket


class MyServer(object):
    def __init__(self, ip, port):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((ip, port))
        self.socket.listen(128)

    def run_forever(self):
        while 1:
            client_socket, client_addr = self.socket.accept()
            data = client_socket.recv(1024).decode('utf8')
            path = ''
            if data:
                path = data.splitlines()[0].split(' ')[1]
                print('请求的路径是{}'.format(path))
            response_header = 'HTTP/1.1 200 OK\n'
            if path == '/login':
                response_body = '<h1 style="color:red">欢迎来到登录页面</h1>'
            elif path == '/register':
                response_body = '<h1 style="color:red">欢迎来到注册页面</h1>'
            elif path == '/':
                response_body = '<h1 style="color:red">欢迎来到首页</h1>'
            else:
                response_header = 'HTTP/1.1 404 Page Not Found\n'
                response_body = '<h1 style="color:red">您要查看的页面不存在！</h1>'
            response_header += 'content-type:text/html;charset=utf8\n'
            response_header += '\n'
            response = response_header + response_body
            client_socket.send(response.encode('utf8'))


s = MyServer('0.0.0.0', 8090)
s.run_forever()

# WSGI服务器

In [ ]:
from wsgiref.simple_server import make_server


#demo_app是一个函数，用来处理用户的请求
#demo_app 需要两个参数，第0个表示环境（电脑的环境；请求路径相关的环境），第1个参数是一个函数，用来返回响应头
#demo_app 需要一个返回值，返回值是一个列表，列表里只有一个元素，是一个二进制，表示返回给浏览器的数据
def demo_app(environ, start_response):
    #environ 是一个字典，保存了很多数据
    path = environ['PATH_INFO']
    #PATH_INFO能够获取到用户的访问路径
    print('path={}'.format(path))
    start_response('200 OK', [('Content-Type', 'text/html;charset=utf8')])
    return ['hello world!'.encode('utf8')]


httpd = make_server('', 8080, demo_app)
sa = httpd.socket.getsockname()
print('Serving HTTP on', sa[0], 'port', sa[1], '...')
#打开电脑的浏览器并输入 http://localhost:8000/xyz?abc
#import webbrowser
#webbrowser.open('http://localhost:8000/xyz?abc')

#处理一次请求
#httpd.handle_request()

#一直运行
httpd.serve_forever()

Serving HTTP on 0.0.0.0 port 8080 ...
path=/


127.0.0.1 - - [06/Mar/2022 22:48:15] "GET / HTTP/1.1" 200 12


path=/favicon.ico


127.0.0.1 - - [06/Mar/2022 22:48:16] "GET /favicon.ico HTTP/1.1" 200 12


In [ ]:
from wsgiref.simple_server import make_server


def demo_app(environ, start_response):
    path = environ['PATH_INFO']
    status_code = '200 OK'
    #状态码：RESTFUL ==> 前后端分离
    #2XX：请求响应成功
    #3XX：重定向
    #4XX：客户端的错误    404：客户端访问了一个不存在的地址    405：请求方式不被允许
    #5XX：服务器的错误
    response = 'Hello world!'
    if path == '/':
        response = '欢迎来到首页'
    elif path == '/test':
        response = '恭喜你喜提习题一份！'
    elif path == '/register':
        response = '欢迎来到注册页面'
    elif path == '/login':
        response = '欢迎来到登录页面'
    else:
        response = '页面走丢了'
        status_code = '404 Not Found'
    start_response(status_code, [('Content-Type', 'text/html;charset=utf8')])
    return [response.encode('utf8')]


httpd = make_server('', 8008, demo_app)
sa = httpd.socket.getsockname()
print('Serving HTTP on', sa[0], 'port', sa[1], '...')
httpd.serve_forever()

Serving HTTP on 0.0.0.0 port 8008 ...


127.0.0.1 - - [06/Mar/2022 23:04:17] "GET / HTTP/1.1" 200 18
127.0.0.1 - - [06/Mar/2022 23:04:18] "GET /favicon.ico HTTP/1.1" 404 15
127.0.0.1 - - [06/Mar/2022 23:04:48] "GET /test HTTP/1.1" 200 30
127.0.0.1 - - [06/Mar/2022 23:05:00] "GET /register HTTP/1.1" 200 24
127.0.0.1 - - [06/Mar/2022 23:05:05] "GET /login HTTP/1.1" 200 24
127.0.0.1 - - [06/Mar/2022 23:05:10] "GET /demo HTTP/1.1" 404 15
